In [2]:
%load_ext autoreload
%autoreload 2

import os
from datetime import datetime
from pathlib import Path

import spatialdata as sd
from loguru import logger
from napari_spatialdata import Interactive

import multiplex_pipeline.core_segmentation.cleaners as cleaners
from multiplex_pipeline.core_preprocessing.controller import (
    PreSegmentationProcessor,
)
from multiplex_pipeline.core_segmentation.controller import (
    SegmentationController,
)
from multiplex_pipeline.core_segmentation.segmenters import (
    InstansegSegmenter,
)
from multiplex_pipeline.utils.utils import load_analysis_settings

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\xarray_schema\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


### Load analysis settings

In [3]:
# load analysis configuration
settings_path = r'C:\BLCA-1_Analysis\analysis_settings_BLCA1.yaml'

settings = load_analysis_settings(settings_path)
settings

{'image_dir': 'R:/CellDive/BLCA-1/BLCA-1_Final',
 'analysis_dir': 'C:/BLCA-1_Analysis',
 'log_dir': WindowsPath('C:/BLCA-1_Analysis/logs'),
 'detection_image': 'BLCA-1_1.0.4_R000_DAPI__FINAL_F.ome.tif',
 'core_info_file_path': WindowsPath('C:/BLCA-1_Analysis/cores.csv'),
 'cores_dir_tif': WindowsPath('C:/BLCA-1_Analysis/temp'),
 'cores_dir_output': WindowsPath('C:/BLCA-1_Analysis/cores'),
 'include_channels': None,
 'exclude_channels': ['008_ECad'],
 'use_markers': None,
 'ignore_markers': ['Antibody1',
  'TNFa',
  'Snail1',
  'SKP2',
  'ProgRc',
  'Plk1',
  'PH3',
  'PDL1',
  'p65',
  'p130',
  'p-p130',
  'p-Cdc6',
  'LAG3',
  'IL-8',
  'HER2',
  'ERa',
  'EpCAM',
  'E2F1',
  'cycD3',
  'cycB2',
  'CDC25C',
  'CD86',
  'CD73',
  'CD69',
  'CD62L',
  'CD56',
  'CD4',
  'CD25',
  'CD19',
  'CD27',
  'CCR7',
  'cCASP3'],
 'segmentation': {'package': 'instaseg',
  'model': 'fluorescence_nuclei_and_cells',
  'kwargs': {'pixel_size': 0.3,
   'resolve_cell_and_nucleus': True,
   'cleanup_fr

### Define the logger

In [4]:
log_file = settings['log_dir'] / f"cores_segmentation_{datetime.now():%Y-%m-%d_%H-%M-%S}.log"

logger.remove()
logger.add(lambda msg: print(msg, end=""))
logger.add(log_file, level="DEBUG", enqueue=True)

2

### Define cores for the analysis

In [5]:
core_dir = Path(settings['analysis_dir']) / 'cores'
path_list = [core_dir / f for f in os.listdir(core_dir)]
path_list.sort()
path_list

[WindowsPath('C:/BLCA-1_Analysis/cores/Core_000.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_001.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_002.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_003.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_004.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_005.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_006.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_007.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_008.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_009.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_010.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_011.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_012.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_013.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_014.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_015.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_016.zarr'),
 WindowsPath('C:/BLCA-1_Analysis/cores/Core_017.

### Setup

In [6]:
# create requested elements
channels = settings['segmentation']['input']
print(f'Segmentation input: {channels}')

# define pre-processor
preseg_processor = PreSegmentationProcessor(normalize = settings['segmentation']['preprocessing']['normalize'],
                                            denoise = settings['segmentation']['preprocessing']['denoise'],
                                            mix = settings['segmentation']['preprocessing']['mix']
                                            )

# define segmenter
if settings['segmentation']['package'] == 'instaseg':
    model = settings['segmentation']['model']
    segmenter = InstansegSegmenter(model_type = model, **settings['segmentation']['kwargs'])
else:
    raise ValueError(f"Segmentation package {settings['segmentation']['package']} to be implemented.")

# define cleaner
cleaner_name = settings['segmentation']['cleaner']
cleaner_class = getattr(cleaners, cleaner_name)
cleaner = cleaner_class()

# define controller
segmentation_controller = SegmentationController(segmenter, cleaner = cleaner, resolution_level = 0, overwrite_mask = True)

# channels used for segmentation
input_channels = [f'{ch}_preseg' for ch in channels]

Segmentation input: ['DAPI', 'HLA1', 'CD44', 'NaKATPase', 'CD11C', 'CD45', 'pCK26', 'pS6', 'HES1']
Model fluorescence_nuclei_and_cells version 0.1.0 already downloaded in c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\instanseg\utils\../bioimageio_models/, loading
Requesting default device: cuda
2025-09-24 12:11:16.496 | INFO     | multiplex_pipeline.core_segmentation.controller:__init__:31 - Initialized SegmentationController with segmenter=<multiplex_pipeline.core_segmentation.segmenters.InstansegSegmenter object at 0x000001FD957FF110>, resolution_level=0, pyramid_levels=3, downscale=2, cleaner=<multiplex_pipeline.core_segmentation.cleaners.BlobCleaner object at 0x000001FD95C40A50> and overwrite_mask=True


In [12]:
torch.cuda.empty_cache()

### Segmentation

In [ ]:
for sd_path in path_list[52:53]:

    print(f"Processing {sd_path.name}")

    # get sdata
    sdata = sd.read_zarr(sd_path)

    # run preprocessing
    sdata = preseg_processor.run(sdata, channels = channels)

    # run segmentation
    segmentation_controller.run(sdata, channels = input_channels, mask_name = settings['segmentation']['output_name'])



version mismatch: detected: RasterFormatV02, requested: FormatV04
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04


Processing Core_053.zarr


version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mi

2025-09-24 12:39:52.242 | INFO     | multiplex_pipeline.core_preprocessing.controller:run_normalize:81 - Applied normalization with percentiles [1, 99.5].
2025-09-24 12:39:53.679 | INFO     | multiplex_pipeline.core_preprocessing.controller:run_normalize:81 - Applied normalization with percentiles [1, 99.5].
2025-09-24 12:39:55.157 | INFO     | multiplex_pipeline.core_preprocessing.controller:run_normalize:81 - Applied normalization with percentiles [1, 99.5].
2025-09-24 12:39:56.617 | INFO     | multiplex_pipeline.core_preprocessing.controller:run_normalize:81 - Applied normalization with percentiles [1, 99.5].
2025-09-24 12:39:58.399 | INFO     | multiplex_pipeline.core_preprocessing.controller:run_normalize:81 - Applied normalization with percentiles [1, 99.5].
2025-09-24 12:39:59.701 | INFO     | multiplex_pipeline.core_preprocessing.controller:run_normalize:81 - Applied normalization with percentiles [1, 99.5].
2025-09-24 12:40:01.256 | INFO     | multiplex_pipeline.core_preproces

### Sneak peak

In [15]:
# refresh the object
sdata_org = sd.read_zarr(path_list[4])
sdata_org

version mismatch: detected: RasterFormatV02, requested: FormatV04
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requeste

SpatialData object, with associated Zarr store: C:\BLCA-1_Analysis\cores\Core_004.zarr
├── Images
│     ├── '53BP1': DataTree[cyx] (1, 7040, 7232), (1, 3520, 3616)
│     ├── 'CD3': DataTree[cyx] (1, 7040, 7232), (1, 3520, 3616)
│     ├── 'CD8a': DataTree[cyx] (1, 7040, 7232), (1, 3520, 3616)
│     ├── 'CD11C': DataTree[cyx] (1, 7040, 7232), (1, 3520, 3616)
│     ├── 'CD11b': DataTree[cyx] (1, 7040, 7232), (1, 3520, 3616)
│     ├── 'CD20': DataTree[cyx] (1, 7040, 7232), (1, 3520, 3616)
│     ├── 'CD31': DataTree[cyx] (1, 7040, 7232), (1, 3520, 3616)
│     ├── 'CD44': DataTree[cyx] (1, 7040, 7232), (1, 3520, 3616)
│     ├── 'CD45': DataTree[cyx] (1, 7040, 7232), (1, 3520, 3616)
│     ├── 'CD45RO': DataTree[cyx] (1, 7040, 7232), (1, 3520, 3616)
│     ├── 'CD68': DataTree[cyx] (1, 7040, 7232), (1, 3520, 3616)
│     ├── 'CD127': DataTree[cyx] (1, 7040, 7232), (1, 3520, 3616)
│     ├── 'CD163': DataTree[cyx] (1, 7040, 7232), (1, 3520, 3616)
│     ├── 'CDC6': DataTree[cyx] (1, 7040, 7232), (1

In [16]:
Interactive(sdata_org)

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\anndata\_core\aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


2025-09-24 10:38:40.208 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-09-24 10:38:40.208 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-09-24 10:38:50.523 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-09-24 10:38:50.531 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-09-24 10:39:07.940 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-09-24 10:39:07.948 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
